If you haven't covered langmem and basic openai-agents-sdk go through first:

1. [Understand LangMem Core APIs](https://colab.research.google.com/drive/1YJNrnQRMgeNTigIuWOfykt-Z5L_DDmsa?usp=sharing)

# **OpenAI Agents SDK with LangMem Memory Tools & InMemory Store**

In [1]:
!pip install -Uq openai-agents langmem langchain-google-genai langmem-adapter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [3]:
from google.colab import userdata
import os

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL = "gemini-2.0-flash"

# import os
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# MODEL = "gpt-4o"

In [4]:
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel, set_tracing_disabled

client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url=BASE_URL
)

model = OpenAIChatCompletionsModel(model=MODEL, openai_client=client)

set_tracing_disabled(disabled=False)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langgraph.store.memory import InMemoryStore

import asyncio
from agents import Agent, Runner

os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

store = InMemoryStore(
      index={
          "dims": 768,
          "embed": GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"),
          # "dims": 1536,
          # "embed": "openai:text-embedding-3-small",
      }
    )


In [7]:
namespace=("assistant", "collection")
print(store.search(namespace))

[]


## 2. OpenAI Agents SDK with Memory Layer

**Quick Note on langmem_adapter:** We can not directly use langchain tool with openai agents sdk. There is difference in how to invoke the tool and much more. Additionally when using completions api with gemini we can not have uuid as argument type for a function used as tool.

To workaround all these tools I created a simple class that takes langmem tools and make them usable with OpenAI Agents SDK. Next I extended it to use langmem features like:
-  Have Dynamic NameSpace Tuples so we can pass userId etc. in runtime using OpenAI Agents SDK Context.
- Inject Store Dynamically or statically
- Handle issues like above uuid one.

You can review the source code here:

https://github.com/mjunaidca/langmem-openaiagents-adapter/blob/main/src/langmem_adapter/openai_agents_sdk.py

In [9]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langmem_adapter import LangMemOpenAIAgentToolAdapter

# Using with InMemory Store and Static NameSpace is simple as follows

adapter = LangMemOpenAIAgentToolAdapter(create_manage_memory_tool(namespace=namespace,store=store))
call_manage_memory_tool = adapter.as_tool()

search_memory_tool_adapter = LangMemOpenAIAgentToolAdapter(create_search_memory_tool(namespace=namespace,store=store))
call_search_memory_tool = search_memory_tool_adapter.as_tool()

In [11]:
# Its a FunctionTool type as in OpenAI Agetns SDK
call_search_memory_tool

FunctionTool(name='search_memory', description='Search your long-term memories for information relevant to your current context.', params_json_schema={'properties': {'query': {'title': 'Query', 'type': 'string'}, 'limit': {'title': 'Limit', 'type': 'integer'}, 'offset': {'title': 'Offset', 'type': 'integer'}, 'filter': {'default': '', 'title': 'Filter', 'type': 'string'}}, 'required': ['query', 'limit', 'offset'], 'title': 'search_memoryArgs', 'type': 'object'}, on_invoke_tool=<bound method LangMemOpenAIAgentToolAdapter._on_invoke_tool of <langmem_adapter.openai_agents_sdk.LangMemOpenAIAgentToolAdapter object at 0x7d48b43635d0>>, strict_json_schema=True)

| **Note:** Here the tools are defined to demonstrate the concent. In production we will use a Persistant Store and have passed all Args with a more better design pattern

In [12]:
agent_system_prompt_memory = """
< Role >
You are Junaids executive assistant. You are a top-notch executive assistant who cares about AI Agents and performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage Junaid's communications and schedule:

1. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
2. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

"""

In [13]:
tools= [
    call_search_memory_tool,
    call_manage_memory_tool
]


In [14]:
import asyncio
import typing
import uuid
from agents import function_tool

agent = Agent(
    name="Assistant",
    instructions=agent_system_prompt_memory,
    model=model,
    tools=tools
)

async def run_example(message: str):

    result = await Runner.run(
        agent,
        message,
    )
    print(result.final_output)


asyncio.run(run_example("Ahmad is my friend"))


OK. I've saved that Ahmad is your friend.



In [15]:
print(store.search(namespace))

[Item(namespace=['assistant', 'collection'], key='4a04c6ab-9aad-49a9-bfb7-82f4da4ee41d', value={'content': "Ahmad is Junaid's friend."}, created_at='2025-03-31T01:03:11.842556+00:00', updated_at='2025-03-31T01:03:11.842563+00:00', score=None)]


In [16]:
asyncio.run(run_example("Who are my friends"))

Okay, it looks like Ahmad is one of your friends.



In [19]:
asyncio.run(run_example("Oh no I meant Muhammad Ahmad not Ahmad is my Friend. Update it without asking any Qs"))

OK. I have updated the memory: Muhammad Ahmad is Junaid's friend.



In [20]:
asyncio.run(run_example("Who are my friends"))

Muhammad Ahmad is Junaid's friend.



In [23]:
asyncio.run(run_example("Sir Zia Ullah Khan is my Mentor"))

Okay, I've made a note that Sir Zia Ullah Khan is your mentor.



In [24]:
asyncio.run(run_example("Who is my Mentor?"))

Sir Zia Ullah Khan is your mentor.

